In [12]:
validationCroiseeGetModel = function(data, partitions, colonneCible, errorRateThreshold = 0.3) {
  errorRate = c()
  #on m\u{fffd}lange les donn\u{fffd}es
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    #Generation des partitions aléatoires
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la pr\u{fffd}cision (le taux d'erreurs)
    #formulaString = paste("modalData_step2$", colonneCible, "~.",sep="")
    #formulaCible= as.formula(formulaString)
    #tree = rpart(formulaCible, modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
      
      
    #Ici on s'assure que notre vecteur "cible" ne contient pas uniquement des 0.
    #Cela peut arriver pour la prediction des dernieres variables (secteur 1, 2 et particulier), qui sont composés de beaucoup de 0
    #Si jamais le modalData_step["variableCible"] contient que des valeurs identiques (ici 0), R nous signale qu'il n'y a aucun interet à créer un modèle et retourne une erreur
    while(sum(modalData_step2[1]) == 0 || sum(testData_step2[1]) == 0){
        #Regenerer les modalData et testData 
        randomData_step2 = data[sample(nrow(data)),]
        testData_step2 = randomData_step2[1:numOfLinesTestCurrentModalData,]
        modalData_step2 = randomData_step2[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
        
    }

    tree = rpart(modalData_step2$X1~., modalData_step2, method="class", control=rpart.control(minsplit=5,cp=0))
    prediction = predict(tree, testData_step2, type="class")
      
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    #formulaString = paste("testData_step2", colonneCible,sep="$")
    #formulaCible2 = as.formula(formulaString)
      
    contingencyTable = table(testData_step2$X1, prediction)
    
    #Calcul de la table de contingence
    #Puis ensuite du taux d'erreur
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la m\u{fffd}me que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  #Si le taux d'erreur est inférieur au seuil fixé (par défaut 0.3)
  #Alors on stocke le modèle dans la listResult et on le return
  if (mean(errorRate) <= errorRateThreshold) {
    listResult = list(mean(errorRate), tree)
    print("Error Rate < 0.3")
    print(mean(errorRate))
    return(listResult)
  }#Sinon on print un message d'erreur et on retourne 0 dans la listResult, se qui permet à la fonction GetModelVariableCible 
    # de savoir qu'elle doit générer un nouveau modèle
    else{
    listResult = list(0, tree)
    print("Error Rate > 0.3, WRONG MODEL")
    print(mean(errorRate))
    return(listResult)
  }
}

#Variable cible : PREDICTION CHIFFRE D'AFFAIRES
GetModelChiffreAffaires = function(data, partitions){
  result = 0
  #Tant que le taux d'erreur n'est pas < seuil
  while(result == 0){
    #print("generating new model ...")
    #On enleve les colonnes inutiles 
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,9)]
    #On les renomme X1 à X30 pour que les nom des colonne des données d'apprentissage correspondent aux noms de colonnes
    # des données de test
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    #On appelle la fonction validationCroiseeGetModel qui retourne une liste avec un entier (0 si erreurRate > seuil, 1 si erreurRate < seuil)
    # et le modèle
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1")
    result = listResult[[1]]
  }
  #Une fois le modèle correct, on l'affiche puis on traite les données de test afin de génération une prediction sur celle-ci
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  #La prediction s'effectue ici, avec le modèle et les données de test en paramètre
  predictionChiffreAffaire = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionChiffreAffaire)
  return (predictionChiffreAffaire)
}

#Variable cible : CAPACITE D'EMPRUNT
GetModelCapaciteEmprunt = function(data, partitions){
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,4,5,6,7,8,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.6)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionCapaciteEmprunt = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionCapaciteEmprunt)
  return (predictionCapaciteEmprunt)
}

GetModelSecteurOne = function(data, partitions){
#Variable cible : SECTEUR 1
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,3,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurOne = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurOne)
  return (predictionSecteurOne)
}

GetModelSecteurTwo = function(data, partitions){
#Variable cible : SECTEUR 2
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,4,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurTwo = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurTwo)
  return (predictionSecteurTwo)
}

GetModelSecteurParticulier = function(data, partitions){
#Variable cible : SECTEUR PARTICULIER
  result = 0
  while(result == 0){
    #print("generating new model ...")
    donneesAppr = data[ , -c(1,2,3,5,6,7,8,9,10)]
    names(donneesAppr) = c("X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
    listResult = validationCroiseeGetModel(donneesAppr, partitions, "X1", 0.5)
    result = listResult[[1]]
  }
  print(listResult[[2]])
  donneesTest = test[ , -c(1,2,3,36,37,38,39)]
  names(donneesTest) = c("X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14","X15","X16","X17","X18","X19","X20","X21","X22","X23","X24","X25","X26","X27","X28","X29","X30","X31")
  predictionSecteurParticulier = predict(listResult[[2]], donneesTest, type="class")
  #print(predictionSecteurParticulier)
  return (predictionSecteurParticulier)
}

In [13]:
#Une fois les fonctions définies, on charge les jeux de données 
apprentissage <- read.table("apprentissage.csv", sep = "," ,header = FALSE)
test <- read.csv("test.csv", sep = "," ,header = FALSE)
# Remplacer les NA par des 0
apprentissage[is.na(apprentissage)]<-0
test[is.na(test)]<-0

#Option permettant de désactiver les warnings pour une meilleure visibilité des résultats
oldw <- getOption("warn")
options(warn = -1)
#On appelle la fonction pour prédire le chiffre d'affaire
predictionChiffreAffaire = GetModelChiffreAffaires(apprentissage, 10)
options(warn = oldw)


[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6166667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8166667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.5166667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.5
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6333333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6666667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.65
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6333333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6666667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.4833333
[1] "Error Rate < 0.3"
[1] 0.2166667
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 17 0 (0.32 0.24 0.04 0.04 0.08 0.08 0.04 0.04 0.04 0.04 0.04)  
   2) X2< 0.5 8  0 0 (1 0 0 0 0 0 0 0 0 0 0) *
   3) X2>=0.5 17 11 17500 (0 0.35 0.059 0.059 0.12 0.12 0.059 0.059 0.059 0.059 0.059)  
     6) X2< 1.5 8  2 17500 (0 0.75 0.12 0.12 0 0 0 0 0 0 0) *
     7) X2>=1.5 9  7 40400 (0 0 0 0 0.22 0.22 0.11 0.11 0.11 0.11 0.11)  


In [14]:
oldw <- getOption("warn")
options(warn = -1)
predictionCapaciteEmprunt = GetModelCapaciteEmprunt(apprentissage, 10)
options(warn = oldw)

[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.95
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.85
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 1
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.85
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.6666667
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.9833333
[1] "Error Rate > 0.3, WRONG MODEL"
[1] 0.8333333
[1] "Error Rate < 0.3"
[1] 0.55
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 13 10000 (0.12 0.04 0.08 0.12 0.48 0.04 0.04 0.08)  
   2) X2< 1.5 15  5 10000 (0.2 0 0.067 0.067 0.67 0 0 0)  
     4) X24>=220.32 3  1 0 (0.67 0 0 0 0.33 0 0 0) *
     5) X24< 220.32 12  3 10000 (0.083 0 0.083 0.083 0.75 0 0 0)  
      10) X9< 1.63 6  3 10000 (0.17 0 0.17 0.17 0.5 0 0 0)  
        20) X15< 533.15 3  2 0 (0.33 0 0.33 0.33 0 0 0 0) *
        21) X1

<strong>
On a remarqué à partir de ces variables cibles "secteurs", que parfois, les jeux de données sont composés uniquement de 0.
Cela induit R en erreur, car il ne comprend pas pourquoi essayer de construire un modèle, sachant que la variable cible à toujours la même valeur : 0.
</strong>

In [15]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurOne = GetModelSecteurOne(apprentissage, 10)
options(warn = oldw)

[1] "Error Rate < 0.3"
[1] 0.15
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 25 3 0 (0.8800000 0.1200000)  
  2) X18< 597.135 20 0 0 (1.0000000 0.0000000) *
  3) X18>=597.135 5 2 1 (0.4000000 0.6000000)  
    6) X6>=423.72 2 0 0 (1.0000000 0.0000000) *
    7) X6< 423.72 3 0 1 (0.0000000 1.0000000) *


In [16]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurTwo = GetModelSecteurTwo(apprentissage, 10)
options(warn = oldw)


[1] "Error Rate < 0.3"
[1] 0.2833333
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 25 4 0 (0.84000000 0.16000000)  
   2) X29< 43.82 23 2 0 (0.91304348 0.08695652)  
     4) X3>=10.125 18 0 0 (1.00000000 0.00000000) *
     5) X3< 10.125 5 2 0 (0.60000000 0.40000000)  
      10) X2>=5.5 2 0 0 (1.00000000 0.00000000) *
      11) X2< 5.5 3 1 1 (0.33333333 0.66666667) *
   3) X29>=43.82 2 0 1 (0.00000000 1.00000000) *


In [17]:
oldw <- getOption("warn")
options(warn = -1)
predictionSecteurParticulier = GetModelSecteurParticulier(apprentissage, 10)
options(warn = oldw)



[1] "Error Rate < 0.3"
[1] 0.1666667
n= 25 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 25 1 0 (0.9600000 0.0400000) *


In [18]:
resultPrediction = data.frame(test[ , -c(1,2,3,35,36,37,38,39)], predictionChiffreAffaire, predictionCapaciteEmprunt, predictionSecteurOne, predictionSecteurTwo, predictionSecteurParticulier)
write.csv(resultPrediction, file = "resultPrediction.csv",row.names=TRUE)